In [1]:

import os 
import cv2
import numpy as np 
import pandas as pd  
import math 
import yaml 
import json 
import torchvision.transforms as transforms 
from PIL import Image 

from dataset.caption_dataset import re_train_dataset
from torchvision import transforms 
from dataset.randaugment import RandomAugment
from PIL import Image 
from torch.utils.data import DataLoader 


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from dataset import create_dataset
from torch.utils.data import DataLoader

train,valid,test = create_dataset('re',config) 
trainloader = DataLoader(train,batch_size=32,shuffle=True)
data = next(iter(trainloader))